<a href="https://colab.research.google.com/github/ShawnTang1709/tibameHW/blob/main/20250113.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request as req
import bs4 as bs
# Step0.
import pandas as pd



# Step1.
table = []

page = 60
for i in range(page):
  print("頁數", i+1)
  url = ("https://tabelog.com/tw/nara/rstLst/{}/?SrtT=rt").format(i+1)
  response = req.urlopen(url)
  html = bs.BeautifulSoup(response)
  r_list = html.find_all("div",{"class":"list-rst__contents"})

  for r in r_list:
    area_name = r.find("a", {"class":"list-rst__rst-name-target"})
    area_href = area_name["href"]
    area_name_text = area_name.text

    area_genre = r.find("div",{"class":"list-rst__area-genre"})
    area_info = r.find("p",{"class":"cpy-pr-title"})
    area_price = r.find_all("span",{"class":"c-rating-v3__val"})
    if area_info == None:
      area_info = "無"
    else:
      area_info = area_info.text.strip()

    area_price_dinner = area_price[0].text
    area_price_lunch = area_price[1].text
    area_dayoff = r.find("span",{"class":"list-rst__holiday-text"})
    area_rate = r.find("span",{"class":"c-rating__val"})
    print(area_name_text)
    print(area_href)
    print("地點/料理樣式:", area_genre.text.strip())
    print("簡介:", area_info)
    print("評分:", area_rate.text)
    print("Dinner Price:", area_price_dinner)
    print("Lunch Price:", area_price_lunch)
    print("公休日期:", area_dayoff.text)
    print("-" * 30)

    # Step2. 每一筆資料就是一個字典
    data = {
        "name(en)": area_name_text,
        "url": area_href,
        "genre": area_genre.text.strip(),
        "intro": area_info,
        "rating": area_rate.text,
        "dinner_price": area_price_dinner,
        "lunch_price": area_price_lunch,
        "dayoff": area_dayoff.text
    }
    table.append(data)
# Step3. 轉成DataFrame儲存
df = pd.DataFrame(table,
          columns = ["name(en)", "url", "genre", "intro", "rating", "dinner_price", "lunch_price", "dayoff"])
df.to_csv("tabelog.csv", encoding = "utf-8")

In [ ]:
import pandas as pd
df = pd.read_csv("tabelog.csv", encoding = "utf-8")
def func(s):
  loc = s.split("/")[0]
  return loc.strip()
df["genre"] = df["genre"].apply(func)
df["genre"].value_counts()

In [ ]:
import pandas as pd
df = pd.read_csv("tabelog.csv", encoding = "utf-8")

def func(s):
  if "大和八木車站" in s:
    return True
  else:
    return False
fil = (df["genre"].apply(func)) & (df["rating"].astype("float") >= 3.5)
df[fil]

In [ ]:
import pandas as pd
df = pd.read_csv("tabelog.csv", encoding = "utf-8")

def func(s, name):
  if name in s:
    return True
  else:
    return False
fil = (df["genre"].apply(func, args=["大和八木車站"])) & (df["rating"].astype("float") >= 3.5)
df[fil]

In [ ]:
intro_all = ("\n".join(df["intro"]))
jieba.analyse.extract_tags(intro_all,allowPOS=["nr", "ns", "nt", "nz"])

In [71]:
def func(s):
  kewards = jieba.analyse.extract_tags(s,
                     allowPOS=["nr", "ns", "nt", "nz"])
  return list(kewards)

df["intro"].apply(func)

,intro
0,[]
1,[]
2,"[充滿希望, 懷石, 正宗]"
3,[]
4,[]
...,...
1195,[]
1196,[]
1197,[]
1198,[]


In [ ]:
practice = pd.DataFrame([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], columns = ["a", "b", "c"], index = ["c", "d", "e"])
#practice

#取行: 字典
# print(type(practice["a"]))
# print(type(practice[["a", "c"]]))

#取列: 1.實際第幾列(iloc) 2.看列標籤(loc)
print(practice)
# print(practice.loc["c"])
# print(practice.iloc[0:2])

#過濾
fil = [True, False, True]
print(practice[fil])

In [ ]:
#店休過濾處理
table = []
for h in df["dayoff"]:
  data = {
      "星期一":True,
      "星期二":True,
      "星期三":True,
      "星期四":True,
      "星期五":True,
      "星期六":True,
      "星期日":True
  }
  check = data.keys()
  for c in check:
    if c in h:
      data[c] = False
  table.append(data)
df2 = pd.DataFrame(table)
total = pd.concat([df,df2], axis = 1)
#過濾: 禮拜日有開，且評分>3.5
fil2 = (total["星期日"] == True) & (total["rating"].astype("float") >= 3.5)
total[fil2]

In [ ]:
import math
#轉換(apply)
df["rating"] = df["rating"].astype("float")
b = round
b(4.87)
def func(state):
  if state == 0:
    return math.gcd
  else:
    return math.lcm
# f = func(1)
# f(3, 6)

func(1)(6,9)

In [ ]:
df["rating"] = df["rating"].astype("float")
df["rating"].apply(round)

#Pandas

```python

1.表格無中生有的SOP
(1).準備一個空list
(2).把每一筆資料準備成一個dict，放進list
(3).轉成DataFrame儲存

2.基本操作
(1).取column
(2).取row
(3).filter
(4).apply
(5).無中生有表格

3.pandas主要型態分兩種
(1).DataFrame: 表格型態，有行有列(2D)
(2).Series: list型態，單行單列(1D)

4.取列(index)
(1).看他實際第幾列(iloc): 把0..2當key
(2).看列標籤(loc): 字典型態，列標籤當作Key
(3).專屬值

5.過濾(filter)
(1).原理:將與列的比數數量相同的True False帶入，對到True會留下，對到False的會丟掉
(2).條件組合 and->& or->

6.轉換(apply)
(1).Series.apply:原有值 -> 轉換方式 -> 新值
(2).轉換方式就是一個函示且不加小括號


"""
函式型態
以前:每個功能都要加小括號()，就算沒有參數也需加入小括號
Everything is an object: 每個東西都有形態，每個型態都有自己的操作
e.g. list:[0][-1] dict:[key]
round是一個型態 round(4.2)中的4.2是這個型態的操作參數
round是步驟集合型態，(4.2)才是把步驟執行得到結果
函式型態: 還沒被打開的錦囊
對函式型態加小括號():把錦囊打開，執行並得到結果
"""
```

#抓出關鍵詞
1.西方語言:英文(字母->詞彙->片語(phrase))
2.東方語言:中文(字->詞彙)

#分詞: 我喜歡吃甜食 -> 我/喜歡/吃/甜食
判斷哪個詞彙在整段話裡最重要:
1.出現次數(TF):高，重要性越高
2.常用程度(IDF):低，重要性越高

#詞彙重要程度:(log = 2的幾次方)
TF-IDF = 出現次數 * (1/常用程度)

#Jieba: 預先統計常用程度
DF = 詞彙出現在幾篇裡面/足夠多篇的文章
1.分詞流程:
(1).利用詞彙庫初步劃分
(2).沒在詞彙庫/混淆狀況:從上下文推測哪種分法最合理
2.更新詞彙資料庫
(1).使用更完整的資料庫
(2).新增客製的詞彙資料庫

In [ ]:
import jieba.analyse

news = """饗饗信義店爆發食物中毒案，累計59人就醫通報，目前該店已恢復營業，針對感染可能，北市衛生局食藥科長林冠蓁表示，個案中有超過7成都有食用過生魚片，且依照潛伏期和相關症狀看來，與諾羅病毒比較相似，但仍需要等待後續報告出爐才能確認。

饗饗食物中毒案累計通報59名個案，昨日核准業者復業，林冠蓁指出，就醫民眾共食的食材包含生魚片、生蠔及干貝，目前已採樣環境檢體4件、人體檢體4件，結果最快1到2周後出爐，而59名通報案中，分別有42人吃生魚片、21人食用干貝、5人食用生蠔，有超過7成有吃過生魚片。



林冠蓁說，根據復業報告中推測，可能是生鮮食材遭受諾羅病毒、病原性大腸桿菌、沙門氏菌等污染，加上目前通報個案的症狀和潛伏期都和諾羅病毒類似，高機率是諾羅病毒感染，但詳細原因仍需待調查報告出爐。

如符合食品中毒病因物質及原因食品，將依違反《食品安全衛生管理法》第15條及第44條處6萬到2億元罰鍰。"""

jieba.analyse.extract_tags(news)

In [ ]:
#設置
import urllib.request as req
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
req.urlretrieve(url, "dict")
jieba.set_dictionary("dict")
jieba.analyse.extract_tags(news)

In [68]:
#使用者自訂義
extra_url = "https://github.com/Elwing-Chou/tibame1224/raw/refs/heads/main/extra.dict.txt"
req.urlretrieve(url, "extra.dict.txt")
jieba.load_userdict("extra.dict.txt")
jieba.analyse.extract_tags(news, allowPOS=["nr", "ns", "nt", "nz"])
"/".join(jieba.cut(news))

'饗/饗/信義/店/爆發/食物中毒/案/，/累計/59/人/就醫/通報/，/目前/該店/已/恢復/營業/，/針對/感染/可能/，/北市/衛生局/食藥/科長/林冠/蓁/表示/，/個案/中有/超過/7/成都/有/食用/過/生魚片/，/且/依照/潛伏期/和/相關/症狀/看來/，/與/諾羅/病毒/比較/相似/，/但/仍/需要/等待/後續/報告/出爐/才能/確認/。/\n/\n/饗/饗/食物中毒/案/累計/通報/59/名/個案/，/昨日/核准/業者/復業/，/林冠/蓁/指出/，/就醫/民眾/共食/的/食材/包含/生魚片/、/生/蠔及/干貝/，/目前/已/採樣/環境/檢體/4/件/、/人/體檢/體/4/件/，/結果/最快/1/到/2/周/後/出爐/，/而/59/名/通報/案中/，/分別/有/42/人/吃/生魚片/、/21/人/食用/干貝/、/5/人/食用/生/蠔/，/有/超過/7/成有/吃過/生魚片/。/\n/\n/ /\n/\n/林冠/蓁/說/，/根據/復業/報告/中/推測/，/可能/是/生/鮮食/材/遭受/諾羅/病毒/、/病原/性/大腸桿菌/、/沙門氏菌/等/污染/，/加上/目前/通報/個案/的/症狀/和/潛伏期/都/和/諾羅/病毒/類似/，/高/機率/是/諾羅/病毒感染/，/但/詳細/原因/仍/需待/調查報告/出爐/。/\n/\n/如/符合/食品/中毒/病因/物質/及/原因/食品/，/將依/違反/《/食品/安全衛生/管理法/》/第/15/條/及第/44/條處/6/萬到/2/億元/罰/鍰/。'